# `00.` Preparation

## `i.` Importing Libraries

In [1]:
import torch
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product
from sklearn.preprocessing import OneHotEncoder
from IPython.display import display, clear_output
from torch.utils.data import Dataset, DataLoader, SequentialSampler, BatchSampler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# for reproducibility
import os
import random

def seed_everything(seed=42):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
seed_everything()

## `ii.` Loading Dataset

In [3]:
ratings_df = pd.read_csv("/kaggle/input/user-movie-dataset/dataset/ratings.csv")
movies_df = pd.read_csv("/kaggle/input/user-movie-dataset/dataset/movies.csv")

df = ratings_df.merge(movies_df,on='movieId',how='left')
df.drop('title',inplace=True,axis=1)
user_len = df['userId'].nunique()
movie_len = df['movieId'].nunique()
print(f"We have {user_len} users")
print(f"We have {movie_len} movies")

We have 610 users
We have 9724 movies


In [4]:
df.head()

,userId,movieId,rating,timestamp,genres
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Comedy|Romance
2,1,6,4.0,964982224,Action|Crime|Thriller
3,1,47,5.0,964983815,Mystery|Thriller
4,1,50,5.0,964982931,Crime|Mystery|Thriller


In [5]:
genres_split = df['genres'].str.get_dummies(sep='|')
df = pd.concat([df, genres_split], axis=1)
df.drop(['genres','(no genres listed)',"timestamp"],inplace=True,axis=1)
df = df.rename(columns={'userId': 'userid','movieId' :'movieid'})
df.head()

,userid,movieid,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0


## `iii.` Data Exploration

### `a.` Explore Null Values


In [6]:
df.isna().sum()

userid         0
movieid        0
rating         0
Action         0
Adventure      0
Animation      0
Children       0
Comedy         0
Crime          0
Documentary    0
Drama          0
Fantasy        0
Film-Noir      0
Horror         0
IMAX           0
Musical        0
Mystery        0
Romance        0
Sci-Fi         0
Thriller       0
War            0
Western        0
dtype: int64

### `b.` Count Unique Items & Users

In [7]:
df.nunique()

userid          610
movieid        9724
rating           10
Action            2
Adventure         2
Animation         2
Children          2
Comedy            2
Crime             2
Documentary       2
Drama             2
Fantasy           2
Film-Noir         2
Horror            2
IMAX              2
Musical           2
Mystery           2
Romance           2
Sci-Fi            2
Thriller          2
War               2
Western           2
dtype: int64

### `c.` Explore Context Variables

### `d.` [Optional] Additional Exploration

In [8]:
df.shape

(100836, 22)

In [9]:
df.describe()

,userid,movieid,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
count,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000,100836.000000
mean,326.127564,19435.295718,3.501557,0.303810,0.239607,0.069301,0.091317,0.387292,0.165427,0.012089,0.415804,0.117359,0.008628,0.072306,0.041106,0.041037,0.076104,0.179737,0.171000,0.262327,0.048187,0.019140
std,182.618491,35530.987199,1.042529,0.459904,0.426846,0.253966,0.288060,0.487134,0.371567,0.109284,0.492862,0.321849,0.092485,0.258994,0.198537,0.198377,0.265165,0.383970,0.376511,0.439902,0.214163,0.137018
min,1.000000,1.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,177.000000,1199.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,325.000000,2991.000000,3.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,477.000000,8122.000000,4.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,610.000000,193609.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


-----------------------------------

# `01.` Data Preprocessing

In order to use the data as an input for the Factorization Machine model, we need it to be in a specific format (i.e., active user group, active movie user group, etc...)

In the following cells, we will perform preprocessing steps to extract required information and prepare the feature vector.

## `i.` Create User Mappings

In order to be able to create the active user column group, create the aggregated history column group, ..., etc., we need the users to have a zero-based sequential indexing.

Hence, we will create a user mapping table (to from the new index to the original id).

In [10]:
df["userid"].nunique()

610

## `ii.` Create Movie Mappings

Similar to the users, in order to be able to create the active movie column group, create the aggregated history column group, ..., etc., we need the users to have a zero-based sequential indexing.

Hence, we will create a movie mapping table (to from the new index to the original id).

In [11]:
movie_mappings =pd.DataFrame({
    'movieid': df["movieid"].unique(),
    'new_id': range(len(df["movieid"].unique()))
}).set_index('movieid')
movie_mappings

,new_id
movieid,
1,0
3,1
6,2
47,3
50,4
...,...
160341,9719
160527,9720
160836,9721


## `iii.` Categorical Columns Encoding

## `iv.` Tagging Transactions into Train and Test Sets

### `#` Select a single transaction per user as a test sample.

**Hint**: Use `cumcount` method (See pandas documentation [here](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.GroupBy.cumcount.html)).

In [12]:
df['Train'] = (df.groupby("userid").cumcount(ascending=False) != 0).replace({True:1, False:0})
df[['userid','movieid','Train']].head()

/tmp/ipykernel_33/2964324798.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Train'] = (df.groupby("userid").cumcount(ascending=False) != 0).replace({True:1, False:0})


,userid,movieid,Train
0,1,1,1
1,1,3,1
2,1,6,1
3,1,47,1
4,1,50,1


In [13]:
df.shape

(100836, 23)

## `v.` Create Aggregated History Matrix

In order to create aggregated user history column group, we need to:
- Create a utility matrix (that indicates the movies that each user has rated **in the training set**).
- Make sure the values filled in the utility matrix don't reflect ratings (i.e. binary, either 1 if rated or 0 if not).
- Normalized the matrix values (we can fo that through dividing by the number of movies rated).

In [14]:
df_copy.movieid.nunique()

NameError: name 'df_copy' is not defined

In [15]:
df_copy.head(200)

NameError: name 'df_copy' is not defined

In [16]:
df_copy = df[df['Train'] == 1][['userid','movieid']].copy() # To avoid changing the original DataFrame
df_copy['ones'] = 1

df_copy['movieid'] = df_copy['movieid'].replace(movie_mappings.new_id.to_dict())
# Replace None with the appropriate values
agg_history = pd.pivot_table(df_copy, values='ones', index='userid', columns='movieid', fill_value=0)
agg_history.shape

(610, 9665)

In [17]:
# Replace None with correct values
# We need to normalize the aggregated history by dividing each value by the sum of all the values in the same row
agg_history_norm = agg_history / agg_history.values.sum(axis=1, keepdims=True)
agg_history_norm.head()

movieid      0         1         2         3         4         5         6     \
userid                                                                          
1        0.004329  0.004329  0.004329  0.004329  0.004329  0.004329  0.004329   
2        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4        0.000000  0.000000  0.000000  0.004651  0.000000  0.000000  0.000000   
5        0.023256  0.000000  0.000000  0.000000  0.023256  0.000000  0.000000   

movieid      7         8         9         10        11        12        13    \
userid                                                                          
1        0.004329  0.004329  0.004329  0.004329  0.004329  0.004329  0.004329   
2        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5        0.023256  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

movieid      14        15        16        17        18        19        20    \
userid                                                                          
1        0.004329  0.004329  0.004329  0.004329  0.004329  0.004329  0.004329   
2        0.000000  0.000000  0.000000  0.000000  0.035714  0.000000  0.000000   
3        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4        0.004651  0.004651  0.004651  0.000000  0.000000  0.000000  0.000000   
5        0.000000  0.000000  0.023256  0.023256  0.000000  0.023256  0.000000   

movieid      21        22        23        24        25        26        27    \
userid                                                                          
1        0.004329  0.004329  0.004329  0.004329  0.004329  0.004329  0.004329   
2        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4        0.000000  0.000000  0.000000  0.004651  0.004651  0.000000  0.000000   
5        0.000000  0.023256  0.000000  0.000000  0.023256  0.000000  0.000000   

movieid      28        29        30        31        32        33        34    \
userid                                                                          
1        0.004329  0.004329  0.004329  0.004329  0.004329  0.004329  0.004329   
2        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3        0.026316  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4        0.000000  0.000000  0.000000  0.004651  0.000000  0.000000  0.004651   
5        0.023256  0.000000  0.000000  0.000000  0.023256  0.023256  0.000000   

movieid      35        36        37        38        39        40        41    \
userid                                                                          
1        0.004329  0.004329  0.004329  0.004329  0.004329  0.004329  0.004329   
2        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4        0.000000  0.004651  0.004651  0.000000  0.000000  0.000000  0.000000   
5        0.023256  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

movieid      42        43        44        45        46        47        48    \
userid                                                                          
1        0.004329  0.004329  0.004329  0.004329  0.004329  0.004329  0.004329   
2        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4        0.000000  0.000000  0.004651  0.000000  0.000000  0.000000  0.000000   
5        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

movieid      49        5

## `vi.` Preparing `OneHotEncoder` for Active Columns Group

In [18]:
df['movieid'] = df['movieid'].replace(movie_mappings.new_id.to_dict())
user_item_pairs = df[['userid','movieid']]
user_item_pairs

,userid,movieid
0,1,0
1,1,1
2,1,2
3,1,3
4,1,4
...,...,...
100831,610,3120
100832,610,2035
100833,610,3121
100834,610,1392


In [19]:
df.movieid.nunique()

9724

In [20]:
active_ohe = OneHotEncoder(sparse_output=False).fit(user_item_pairs)

In [21]:
active_ohe.fit_transform(user_item_pairs).shape

(100836, 10334)

### `#` Testing with a sample DataFrame

In [22]:
sample_df = pd.DataFrame({'userid': [1], 'movieid': [5], 'any_other_features': ['values']})
sample_df

,userid,movieid,any_other_features
0,1,5,values


In [23]:
active_ohe.get_feature_names_out()

array(['userid_1', 'userid_2', 'userid_3', ..., 'movieid_9721',
       'movieid_9722', 'movieid_9723'], dtype=object)

In [24]:
pd.DataFrame(
    active_ohe.transform(
        sample_df[active_ohe.feature_names_in_]
        ),
    columns=active_ohe.get_feature_names_out()
    )

,userid_1,userid_2,userid_3,userid_4,userid_5,userid_6,userid_7,userid_8,userid_9,userid_10,userid_11,userid_12,userid_13,userid_14,userid_15,userid_16,userid_17,userid_18,userid_19,userid_20,userid_21,userid_22,userid_23,userid_24,userid_25,userid_26,userid_27,userid_28,userid_29,userid_30,userid_31,userid_32,userid_33,userid_34,userid_35,userid_36,userid_37,userid_38,userid_39,userid_40,userid_41,userid_42,userid_43,userid_44,userid_45,userid_46,userid_47,userid_48,userid_49,userid_50,userid_51,userid_52,userid_53,userid_54,userid_55,userid_56,userid_57,userid_58,userid_59,userid_60,userid_61,userid_62,userid_63,userid_64,userid_65,userid_66,userid_67,userid_68,userid_69,userid_70,userid_71,userid_72,userid_73,userid_74,userid_75,userid_76,userid_77,userid_78,userid_79,userid_80,userid_81,userid_82,userid_83,userid_84,userid_85,userid_86,userid_87,userid_88,userid_89,userid_90,userid_91,userid_92,userid_93,userid_94,userid_95,userid_96,userid_97,userid_98,userid_99,userid_100,userid_101,userid_102,userid_103,userid_104,userid_105,userid_106,userid_107,userid_108,userid_109,userid_110,userid_111,userid_112,userid_113,userid_114,userid_115,userid_116,userid_117,userid_118,userid_119,userid_120,userid_121,userid_122,userid_123,userid_124,userid_125,userid_126,userid_127,userid_128,userid_129,userid_130,userid_131,userid_132,userid_133,userid_134,userid_135,userid_136,userid_137,userid_138,userid_139,userid_140,userid_141,userid_142,userid_143,userid_144,userid_145,userid_146,userid_147,userid_148,userid_149,userid_150,userid_151,userid_152,userid_153,userid_154,userid_155,userid_156,userid_157,userid_158,userid_159,userid_160,userid_161,userid_162,userid_163,userid_164,userid_165,userid_166,userid_167,userid_168,userid_169,userid_170,userid_171,userid_172,userid_173,userid_174,userid_175,userid_176,userid_177,userid_178,userid_179,userid_180,userid_181,userid_182,userid_183,userid_184,userid_185,userid_186,userid_187,userid_188,userid_189,userid_190,userid_191,userid_192,userid_193,userid_194,userid_195,userid_196,userid_197,userid_198,userid_199,userid_200,userid_201,userid_202,userid_203,userid_204,userid_205,userid_206,userid_207,userid_208,userid_209,userid_210,userid_211,userid_212,userid_213,userid_214,userid_215,userid_216,userid_217,userid_218,userid_219,userid_220,userid_221,userid_222,userid_223,userid_224,userid_225,userid_226,userid_227,userid_228,userid_229,userid_230,userid_231,userid_232,userid_233,userid_234,userid_235,userid_236,userid_237,userid_238,userid_239,userid_240,userid_241,userid_242,userid_243,userid_244,userid_245,userid_246,userid_247,userid_248,userid_249,userid_250,userid_251,userid_252,userid_253,userid_254,userid_255,userid_256,userid_257,userid_258,userid_259,userid_260,userid_261,userid_262,userid_263,userid_264,userid_265,userid_266,userid_267,userid_268,userid_269,userid_270,userid_271,userid_272,userid_273,userid_274,userid_275,userid_276,userid_277,userid_278,userid_279,userid_280,userid_281,userid_282,userid_283,userid_284,userid_285,userid_286,userid_287,userid_288,userid_289,userid_290,userid_291,userid_292,userid_293,userid_294,userid_295,userid_296,userid_297,userid_298,userid_299,userid_300,userid_301,userid_302,userid_303,userid_304,userid_305,userid_306,userid_307,userid_308,userid_309,userid_310,userid_311,userid_312,userid_313,userid_314,userid_315,userid_316,userid_317,userid_318,userid_319,userid_320,userid_321,userid_322,userid_323,userid_324,userid_325,userid_326,userid_327,userid_328,userid_329,userid_330,userid_331,userid_332,userid_333,userid_334,userid_335,userid_336,userid_337,userid_338,userid_339,userid_340,userid_341,userid_342,userid_343,userid_344,userid_345,userid_346,userid_347,userid_348,userid_349,userid_350,userid_351,userid_352,userid_353,userid_354,userid_355,userid_356,userid_357,userid_358,userid_359,userid_360,userid_361,userid_362,userid_363,userid_364,userid_365,userid_366,userid_367,userid_368,userid_369,userid_370,userid_371,userid_372,userid_373,user

## `vii.` Putting it all together

### `#` Concatenate the Columns Created Independently

Preview the New Dataframe Containing:
- Mapped `userid` and `itemid`
- Encoded Categorical Columns `Time`, `Location`, `Companion`
- Training Data Tag Column
- Target Column `rating`

In [25]:
df.head()

,userid,movieid,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Train
0,1,0,4.0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,1,1,4.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,1,2,4.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,1,3,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1
4,1,4,5.0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1


In [26]:
new_df = df
new_df

,userid,movieid,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Train
0,1,0,4.0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,1,1,4.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,1,2,4.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,1,3,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1
4,1,4,5.0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,3120,4.0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1
100832,610,2035,5.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
100833,610,3121,5.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
100834,610,1392,5.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1


### `#` Preview the Normalized Aggregated History Matrix

### `#` Split Data into Training and Test Dataframes

**Hint**: Use `Train` Column

In [27]:
train = new_df[new_df["Train"]==1]
test = new_df[new_df["Train"]==0]
print('The dimensions of the training subset: ', train.shape[0], 'X', train.shape[1])
print('The dimensions of the test subset: ', test.shape[0], 'X', test.shape[1])

The dimensions of the training subset:  100226 X 23
The dimensions of the test subset:  610 X 23


In [28]:
test.head()

,userid,movieid,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Train
231,1,231,5.0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
260,2,258,5.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
299,3,290,0.5,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0
515,4,459,5.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
559,5,36,3.0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0


-----------------------

# `02.` Torch Dataset and DataLoader

## `i.` Create Customized Dataset Class

Make sure the `__getitem__` method does the following before returning the data:
- One Hot Encode the `userid` and `itemid` to create active user and active movie column groups respectively.
- Concatenates the redundent rows -corresponding to the `userid` column- from the normalized aggregated history matrix.
- Convert the data into sparse format (to avoid exceeding memory capacity).

**Note**: The Dataset will take an input of:
- User Ids: This is the set of user ids that will be used to split the batches
- Features Dataframe: Contains the data features (except for the aggregated history columns)
- Normalized Aggregated History Matrix
- Active Encoder (i.e. one-hot encoder fit on `userid` and `itemid`)
- Boolean variable indicating whether this dataset is for training/validation (having ground truth) or recommendation (no ground truth)

In [29]:
class DPMovieDataset(Dataset):
  def __init__(self,  data: pd.DataFrame, agg_hist: pd.DataFrame, active_encoder: OneHotEncoder, recommendation: bool=False):
    self.user_ids = data.userid.values
    self.data = data
    self.agg_hist = agg_hist
    self.active_encoder = active_encoder
    self.recommendation = recommendation

  def __len__(self):
    #print("user_ids",self.data.userid.nunique())
    return self.data.userid.nunique()+1

  def __getitem__(self, idx):  
    #print("idx_list",idx)
    batch_data = self.data[self.data["userid"].isin(idx)] # Select the rows corresponding to the list of user indices `idx` from self.data dataframe
    columns_to_drop = ["userid","movieid","Train","rating"]
    columns_to_drop = [col for col in columns_to_drop if col in batch_data.columns]
    cat_cols = batch_data.drop(columns_to_drop,axis=1) # From batch_data extract only the one-hot encoded categorical columns (i.e., drop id, target, and any other columns from batch_data)
    agg_history = batch_data[['userid']].merge(self.agg_hist, left_on='userid', right_index=True) # Get the aggregated history for each selected transaction using merge
    #print(batch_data.head())
    if len(agg_history)==0:
        print("take care 0 length of agg_history")
        
    active_groups = self.active_encoder.transform(batch_data[self.active_encoder.feature_names_in_]) # Use active_encoder to generate the active columns for user/item pairs available in batch_data
    #print("active_groups",active_groups)
    features = torch.from_numpy(np.hstack((active_groups, agg_history.values, cat_cols.values))) # Concatenate the processed columns together horizontally

    if not self.recommendation:
      targets = batch_data['rating']
      return features, targets
    else:
      return features

## `ii.` Instantiate Train and Test Objects from the Custom Dataset



In [30]:
print("train",train.shape)
print("agg_history_norm",agg_history_norm.shape)

train (100226, 23)
agg_history_norm (610, 9665)


In [31]:
train.userid.values

array([  1,   1,   1, ..., 610, 610, 610])

In [32]:
dataset_train = DPMovieDataset( train, agg_history_norm, active_ohe)
dataset_test = DPMovieDataset( test, agg_history_norm, active_ohe)

## `iii.` Create the DataLoader for Train and Test Datasets

**Note**: `BatchSampler` is used to select the mini batch at one go; since the `DataLoader` samples the batch items one by one
 causing an error when concatenating (because each item in our case represents a user with different transaction count; meaning the items don't have
 consistent shapes).


**Note**: `BatchSampler` can have either another sampler or list of indices to select from. In our case, we used `SequentialSampler` which generates selections from 0 to M (number of users) in subsets of batch_size.

In [33]:
dataloader_train = DataLoader(dataset_train,
                              sampler=BatchSampler(SequentialSampler(dataset_train), batch_size=10, drop_last=False),
                              batch_size=None)

dataloader_test = DataLoader(dataset_test,
                              sampler=BatchSampler(SequentialSampler(dataset_test), batch_size=10, drop_last=False),
                              batch_size=None)

----------------------------------------------

# `03.` Torch Model Implementation

## `#` Model Equation Review

The 2-way factorization machine can be defines as:
$$\hat{y}(x) = w_0 + \sum_{i=1}^{n}w_i x_i + \sum_{i=1}^{n} \sum_{j=i+1}^{n} \langle \textbf v_i, \textbf v_j \rangle x_i x_j$$

<br>
The first term is just a linear model (i.e. can be replaced with a linear layer). The interaction part though can be reformulated into:

$$\sum_{i=1}^{n} \sum_{j=i+1}^{n} \langle \textbf v_i, \textbf v_j \rangle x_i x_j =
\frac{1}{2} \sum_{f=1}^{k} \Big( \big(\sum_{i=1}^{n} v_f^{(i)} x_i \big)^2 - \sum_{i=1}^{n}v_f^{(i) 2} x_i^2 \Big)
$$

<br>

If we define $X$ and $V$ as follows:

<br>

$$
X = \begin{bmatrix}
x_1^{(1)} & \dots & x_n^{(1)}\\
 \vdots \ & \ddots \ & \vdots \\
x_1^{(M)} & \dots & x_n^{(M)} \\
\end{bmatrix}, \space \space
V = \begin{bmatrix}
v_1^{(1)} & \dots & v_k^{(1)}\\
 \vdots \ & \ddots \ & \vdots \\
v_1^{(n)} & \dots & v_k^{(n)} \\
\end{bmatrix}
$$

<br>

where $M$ is the number of transactions, $n$ is the number of features, and $k$ is the hyperparameter defining the latent space dimensions.

<br><br>
Then the interaction term can be put in the following vectorized form:

$$\sum_{i=1}^{n} \sum_{j=i+1}^{n} \langle \textbf v_i, \textbf v_j \rangle x_i x_j =
\frac{1}{2} \sum^{M} \Big( \big(XV\big)^{2} - \big(X^{2}V^{2}\big) \Big)
$$

<br>

where $\sum\limits^{M}$ represents a row-wise summation.

## `i.` Create the Model Class

Implement Factorization Machines using PyTorch Class Model:
- Linear Layer: Represents the variable strengths
- V Matrix: Latent Representation Matrix for each variable
- Implement the FM calculations in the `forward` method

In [34]:
class FactorizationMachine(torch.nn.Module):
  def __init__(self, n, k, bias=False):
    super(FactorizationMachine, self).__init__()
    self.n = n
    self.k = k
    self.linear = torch.nn.Linear(self.n, 1, bias)
    self.V = torch.nn.Parameter(torch.randn(n,k)) # Creating the latent matrix V of size (n X k) and initializing it with random values

  def forward(self, x_batch):
    x_batch = x_batch.float()
    part_1 = torch.sum(torch.mm(x_batch, self.V) ** 2, dim=1, keepdim=True)  # perform the first part of the interaction term: row-wise-sum((XV)^2)
    part_2 = torch.sum(torch.mm(x_batch ** 2, self.V ** 2), dim=1, keepdim=True) # perform the second part of the interaction term: row-wise-sum((X)^2 * (V)^2))
    inter_term = 0.5 * (part_1 - part_2) # Put the interaction term parts together (refer to the equations above)
    #print("inter_term",inter_term.shape)
    var_strength =  self.linear(x_batch).squeeze(dim=0) # Perform the linear part of the model equation (refer to the demo notebook on how to use layers in pytorch models)
    #print("var_strength",var_strength.shape)
    #print("Data outputted from forward model ",(var_strength + inter_term).shape)
    return var_strength + inter_term

## `ii.` Instantiate a Model Object

As for the hyperparameters:
- `n`: No. of features in the final feature vector
- `k`: No. of latent features in the `V` matrix

**Note**: You may try $k=5$ and observe the results.

In [35]:
# Hint: This is how you can check the no. features in our dataset
total=0
for x, y in dataloader_train:
    print(x.shape)
    total+=x.shape[0]
    print("total",total)

torch.Size([1110, 20019])
total 1110
torch.Size([1848, 20019])
total 2958
torch.Size([1858, 20019])
total 4816
torch.Size([700, 20019])
total 5516
torch.Size([1547, 20019])
total 7063
torch.Size([1608, 20019])
total 8671
torch.Size([2926, 20019])
total 11597
torch.Size([861, 20019])
total 12458
torch.Size([1520, 20019])
total 13978
torch.Size([1223, 20019])
total 15201
torch.Size([1897, 20019])
total 17098
torch.Size([1534, 20019])
total 18632
torch.Size([1061, 20019])
total 19693
torch.Size([1251, 20019])
total 20944
torch.Size([1184, 20019])
total 22128
torch.Size([939, 20019])
total 23067
torch.Size([1354, 20019])
total 24421
torch.Size([1350, 20019])
total 25771
torch.Size([1899, 20019])
total 27670
torch.Size([1065, 20019])
total 28735
torch.Size([1098, 20019])
total 29833
torch.Size([2000, 20019])
total 31833
torch.Size([1673, 20019])
total 33506
torch.Size([1835, 20019])
total 35341
torch.Size([1830, 20019])
total 37171
torch.Size([544, 20019])
total 37715
torch.Size([1076, 2001

In [36]:
model = FactorizationMachine(n=20019, k=5)

------------------------

# `04.` Training the Model

## `i.` Write a Function to Run a Single Step of Training

Parameters:
- Model [torch model]
- Optimizer [torch.optim optimizer]
- Faetures [matrix]
- Targets [vector]
- Train [bool]: Whether this is a training or an evaluation step

In [37]:
def model_step(mode, x, y=None, optimizer=None, train=True):
  if train: # If we're in training phase, then zero the gradients and make sure the model is set to train
    model.train()
    optimizer.zero_grad()
  else: # If we're in evaluation phase, then make sure the model is set to eval
    model.eval()

  with torch.set_grad_enabled(train): # Either to perform the next lines with gradient tracing or not
    pred =  model(x) # Get the model output from x
    #print("pred shape",pred.shape)
    pred = pred.reshape(pred.shape[0], ) # Flatten the prediction values

    y = torch.from_numpy(y.values.reshape(y.shape[0], )).float()

    criterion=torch.nn.MSELoss() # Define the criterion as MSELoss from torch
    loss = criterion(pred, y)

    if train:
      loss.backward()
      optimizer.step()

  return loss

## `ii.` Write a Function to Simulate the Training Loop of the Model

Parameters:
- Model [torch model]
- Training Dataloader [torch DataLoader]
- Test/Eval Dataloader [torch DataLoader]
- Learning Rate [float]
- Weight Decay [float]
- Epochs [int]
- Eval Step [int]


In [38]:
def train_loop(model, train_loader, eval_loader, lr, w_decay, epochs, eval_step):
    step = 0
    """ Defining our optimizer """
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=w_decay)
    epochs_l, steps, t_losses, v_losses = [], [], [], []

    epochs_tqdm = tqdm(range(epochs), desc='Training in Progress', leave=True)
    for epoch in epochs_tqdm:
        for x, y in train_loader:
              #print("x.shape",x.shape)
              #print("y.shape",y.shape)
              loss_batch = model_step(model, x, y, optimizer, train=True)
              step +=1
              if step % eval_step == 0:
                train_loss = loss_batch
                val_loss = 0
                for x, y in eval_loader:
                    val_loss += model_step(model, x, y, train=False)
                epochs_l.append(epoch+1)
                steps.append(step)
                t_losses.append(train_loss.detach().numpy())
                v_losses.append(val_loss.detach().numpy())
                clear_output(wait=True)
                display(pd.DataFrame({'Epoch': epochs_l, 'Step': steps, 'Training Loss': t_losses, 'Validation Loss': v_losses}))


In [ ]:
train_loop(model, dataloader_train, dataloader_test, lr=0.0004, w_decay=0.003, epochs=100, eval_step=60)

,Epoch,Step,Training Loss,Validation Loss
0,1,60,115.99067,2419.7659
1,2,120,19.501287,383.26163
2,3,180,2.5290055,223.66045
3,4,240,1.9174552,146.55612
4,5,300,5.320186,145.2071
5,6,360,2.4122233,145.52124
6,7,420,1.2051451,158.28079
7,8,480,1.1840739,147.73254
8,9,540,1.1874732,130.74153
9,10,600,1.3597522,140.87997


In [ ]:
import pickle
with open('my_model.pkl', 'wb') as f:
    pickle.dump(model, f)

-------------------------

# `05.` Recommendation



## `i.` Evaluating Model on Test Transaction

Evaluate the model on `userid`$=1053$ from the test dataset

**Note**: Make sure this transaction is not in your training set (if it's, then evaluate the test transaction for this user)

In [46]:
float(model(dataset_test.__getitem__([86])[0]))

2.600958824157715

In [47]:
model.eval()
with torch.no_grad():
  print(f'Predicted rating for User of interest: {model(dataset_train.__getitem__([86])[0])}') # Get the model output on the user of interest after running the previous cell to now their new_id
  print(f'Actual Rating: {float(test[test["userid"]==86]["rating"].values)}') # Extract the actual rating for the user of interest from dataset_test Dataset object

Predicted rating for User of interest: tensor([[3.1977],
        [2.6837],
        [3.4199],
        [3.4913],
        [3.1898],
        [1.8823],
        [3.1871],
        [2.9781],
        [3.3453],
        [3.2097],
        [2.9288],
        [2.8466],
        [2.6769],
        [2.0291],
        [2.8120],
        [3.3926],
        [3.2281],
        [3.0228],
        [3.0075],
        [2.6666],
        [3.0996],
        [3.0972],
        [3.3697],
        [2.4380],
        [2.6354],
        [3.3232],
        [2.4296],
        [2.1075],
        [2.8542],
        [2.9834],
        [2.0306],
        [3.3023],
        [3.1113],
        [2.6389],
        [2.6251],
        [3.2454],
        [3.0083],
        [2.9675],
        [3.3093],
        [3.1713],
        [2.4472],
        [3.1106],
        [2.8402],
        [3.0088],
        [3.0951],
        [2.0152],
        [2.8797],
        [2.8064],
        [3.1176],
        [2.6504],
        [3.2898],
        [3.1030],
        [3.1237],
       

/tmp/ipykernel_33/2310177497.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f'Actual Rating: {float(test[test["userid"]==86]["rating"].values)}') # Extract the actual rating for the user of interest from dataset_test Dataset object


In [ ]:
model.eval()
with torch.no_grad():
  print(f'Predicted rating for User of interest: {None}') # Get the model output on the user of interest after running the previous cell to now their new_id
  print(f'Actual Rating: {None}') # Extract the actual rating for the user of interest from dataset_test Dataset object

## `ii.` Recommending New Items

Create a dataframe containing dummy transactions involving user `userid`$=1053$ and movies they didn't rate before.

### `#` Get a list of movies not rated before

In [ ]:
# Replace None with the new_id of the user
items_our_user_rated = (train[train.userid==86].movieid).unique().tolist()
items_our_user_rated.extend((test[test.userid==86].movieid).unique().tolist())

In [ ]:
items_our_user_can_rate = movie_mappings[~movie_mappings.new_id.isin(items_our_user_rated)].new_id.tolist()

In [ ]:
print(f'Number of unique items user of interest rated is {len(items_our_user_rated)}')
print(f'Number of unique items that can be recommended to user of interest is {len(items_our_user_can_rate)}')

print(f'Preview of the item list:\n\t{items_our_user_can_rate[:5]}')

In [ ]:
print(f'Number of unique items user of interest rated is {len(items_our_user_rated)}')
print(f'Number of unique items that can be recommended to user of interest is {len(items_our_user_can_rate)}')

print(f'Preview of the item list:\n\t{items_our_user_can_rate[:5]}')

### `#` Generate a list of all possible pairs of movies with the different contexual features
- `Time`: [Time_Weekday, Time_Weekend]
- `Location`: [Location_Cenima, Location_Home]
- `Companion`: [Companion_Alone, Companion_Family, Companion_Partner]

**Hint**: you can use `itertools.product` function (Not familar? see [Explanation](https://www.geeksforgeeks.org/python-itertools-product/))

In [ ]:
possible_pairs = product(items_our_user_can_rate)

### `#` Create a `DataFrame` where each row represents a possible pair

**Hint 1**: DataFrames can be created from a list of dictionaries (refer to an example [here](https://stackoverflow.com/questions/20638006/convert-list-of-dictionaries-to-a-pandas-dataframe))

**Hint 2**: [List comprehensions](https://www.w3schools.com/python/python_lists_comprehension.asp) can be used with [f-strings](https://www.geeksforgeeks.org/formatted-string-literals-f-strings-python/) to create a list of dictionaries where each row has $1$s in certain columns and $0$s elsewhere

In [ ]:
df.head()

In [ ]:
def take_user_film_id_make_prediction(user_id,movieid):
    recommendation_df=df[df["movieid"]==movieid].head(1).reset_index(drop=True)
    recommendation_df.loc[0,"userid"]=user_id
    recommendation_df
    recommendation_df=recommendation_df.drop(columns=["rating","Train"])
    recommendation_dataset = DPMovieDataset( recommendation_df, agg_history_norm, active_ohe, recommendation=True)
    recommendation_dataloader = DataLoader(recommendation_dataset,
                                        sampler=BatchSampler([user_id], batch_size=1, drop_last=False),
                                        batch_size=None)
    rating_predictions = []
    with torch.no_grad():
      model.eval()
      for batch in recommendation_dataloader:
        rating_predictions.append(model(batch).item())
    
    return rating_predictions

In [ ]:
 take_user_film_id_make_prediction(120,30)

In [ ]:
recommendation_df=df[df["movieid"]==50].head(1).reset_index(drop=True)
recommendation_df.loc[0,"userid"]=22
recommendation_df

In [ ]:
data_column_order = train.drop(columns=['Train', 'rating']).columns.tolist()
data_column_order

### `#` Instantiate objects from the `DPMovieDataset` and `DataLoader` to prepare our dummy transactions for the model

**Note**: `batch_size` in the recommendation phase can be 1 so that it's easier to determine which predictions belong to which user (in case of recommending to multiple users at once)

**Hint**: `BatchSampler` can take another sampler (e.g. `SequentialSampler` as we used before) or an iterable (e.g. list of indices)

In [ ]:
# Replace None with the new_id of the user of interest

users_to_recommend = [86]

In [ ]:
recommendation_dataset = DPMovieDataset(np.array([users_to_recommend]), recommendation_df, agg_history_norm, active_ohe, recommendation=True)
recommendation_dataloader = DataLoader(recommendation_dataset,
                                        sampler=BatchSampler(users_to_recommend, batch_size=1, drop_last=False),
                                        batch_size=None)

### `#` Rating Prediction

In [ ]:
rating_predictions = []
with torch.no_grad():
  model.eval()
  for batch in recommendation_dataloader:
    rating_predictions.append(model(batch))

### `#` Add `predicted_rating` column to `recommendation_df` and fill it with the corresponding $<user, movie>$ pairs ratings

In [ ]:
for user, ratings in zip(users_to_recommend, rating_predictions):
  recommendation_df.loc[recommendation_df[recommendation_df.userid==user].index, 'predicted_rating'] = ratings.numpy()

### `#` Producing Top-N lists based on the contexual features

Let's take $N=5$

In [ ]:
possible_contexts = product(['Time_Weekday', 'Time_Weekend'], ['Location_Cinema', 'Location_Home'], ['Companion_Alone', 'Companion_Family', 'Companion_Partner'])

In [ ]:
top5_lists = pd.DataFrame([{'user': t[0],
               'time_context': t[1][0].replace('Time_', ''),
               'location_context': t[1][1].replace('Location_', ''),
               'companion_context': t[1][2].replace('Companion_', ''),
               'top5': recommendation_df[(recommendation_df.userid==t[0])\
                                         &(recommendation_df[t[1][0]]==1)\
                                         &(recommendation_df[t[1][1]]==1)\
                                         &(recommendation_df[t[1][2]]==1)].sort_values('predicted_rating', ascending=False).itemid.values[:5]\
               } for t in product(users_to_recommend, possible_contexts)])

In [ ]:
top5_lists

In [ ]:
top5_lists

### `#` [Additional] Request Movie Titles from `OMDb` API

**Note**: [Open Movie Database (OMDb)](https://www.omdbapi.com/) can be used to retrieve movie information using `imdb` id (e.g. $tt3896198$)

**Hint**: you can use `requests` to send requests to the API (see [explanation](https://www.geeksforgeeks.org/python-requests-tutorial/))

In [ ]:
film_id =  movie_mappings[movie_mappings["new_id"]==76].index.values[0]
print("film_id",film_id)

In [ ]:
# Implement the function to take new_id value map it back to original id and then query the API and return the movie title
def get_titles(index_id):
   
    film_id =  movie_mappings[movie_mappings["new_id"]==index_id].index.values[0]
    url = f'https://www.omdbapi.com/?i={film_id}&apikey=958d0fe'
    response = requests.get(url)
    #print(response.text)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        # Get the film title
        film_title = data.get('Title')
        if film_title:
            #print(f'Film Title: {film_title}')
            return film_title
        else:
            #print('Film not found.')
            return None
    else:
        print('Failed to retrieve data from OMDB API.')
        return None

In [ ]:
top5_lists['top5_titles'] = top5_lists.top5.apply(lambda id_list: [get_titles(id_) for id_ in id_list])

In [ ]:
top5_lists

In [ ]:
top5_lists

### `#` [Bonus] Use the `OMDb` API to add more auxiliary features (e.g. genre, year, etc.)

----------------------------------------------

$$ Wish \space you \space all \space the \space best \space ♡ $$
$$ Abdelrahman \space Eid $$